In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from typing import *

We want to add cumulative timescales to the summary. 

In [2]:
protein='1fme'
batch_num = 1
raw_path = f'../{protein}/raw_batch{batch_num}.h5'
summary_path = f'../{protein}/summary_batch{batch_num}.h5'
timescales = pd.DataFrame(pd.read_hdf(raw_path, key='timescales'))

timescales.reset_index(inplace=True)
timescales.rename(mapper={0: 'value'}, axis=1, inplace=True)

In [3]:
timescales.sort_values(by=['hp_ix', 'bs_ix', 'lag', 'process'], inplace=True)
# eigenvalues['ratio'] = eigenvalues.groupby(['hp_ix', 'bs_ix', 'lag', 'process'])['value'].diff()

In [4]:
timescales.head()

,hp_ix,bs_ix,lag,process,value
2563210,0,0,1,2,875.664713
2563211,0,0,1,3,286.509421
2563212,0,0,1,4,263.436623
2563213,0,0,1,5,158.381910
2563214,0,0,1,6,154.196723


In [6]:
timescales['cum_ts'] = timescales.groupby(['hp_ix', 'bs_ix', 'lag'])['value'].cumsum()

In [7]:
print(timescales.shape[0])
timescales.dropna(inplace=True)
timescales.shape[0]

5812692


5812692

In [9]:
summary = timescales.groupby(['hp_ix', 'lag', 'process'])['cum_ts'].agg(
            median=lambda x: np.nanquantile(x, 0.5),
            lb=lambda x: np.nanquantile(x, 0.025),
            ub=lambda x: np.nanquantile(x, 0.975),
            count=lambda x: x.shape[0]-x.isna().sum()
)
summary.reset_index(inplace=True)

In [10]:
summary.to_hdf(summary_path, key='cumulative_ts')